In [1]:
import numpy as np
import cv2 as cv
import trimesh
import matplotlib.pyplot as plt


from visualisation import visualize_results, visualize_results_mesh
from check_visibility import get_visible_vertices, load_image_info, prepare_camera_and_pose_data, load_and_prepare_mesh
from check_visibility import IMG_FOLDER_PATH, IMG_FORMAT, POSES_FOLDER, MESH_PATH
import cv2 as cv


from read_metashape_cameras_xml import create_metashape_cameras, CAMERAS_XML_PATH, OUTPUT_MAPPING_PATH
from create_poses_data import  split_metashape_poses_into_jsons, MAPPING_PATH, OPHIKAPPA_POSES_PATH, OUTPUT_FOLDER_PATH
import json


### Prepare data

In [2]:
prepare_data = False
if prepare_data: 
    images_to_cameras = create_metashape_cameras(CAMERAS_XML_PATH)
    with open(OUTPUT_MAPPING_PATH, "w") as f:
            json.dump(images_to_cameras, f, indent=4)
    poses_nr = split_metashape_poses_into_jsons(
            MAPPING_PATH, OPHIKAPPA_POSES_PATH, OUTPUT_FOLDER_PATH
        )

In [4]:
image_name = 'DJI_20240417190632_0129_Z'
pose_info, camera_info = load_image_info(image_name)
camera_matrix, distortion_coefficients, rotation, center, img_height, img_width = prepare_camera_and_pose_data(camera_info, pose_info)
pier, pier_cutted_ = load_and_prepare_mesh(MESH_PATH, camera_matrix, rotation, center, img_height, img_width)

visible_indices_orig, visible_indices = get_visible_vertices(image_name)


Total vertices 391
Projected vertices 275
Ray hits 230
Visible 92
Total vertices original 12319


In [5]:
visualize_results_mesh(pier_cutted_, visible_indices, rotation, center)

# Use 10 images

In [6]:
images_list =   ['DJI_20240418185755_0185_Z',
                'DJI_20240418185757_0186_Z',
                'DJI_20240418185759_0187_Z',
                'DJI_20240418185801_0188_Z',
                'DJI_20240418185803_0189_Z',
                'DJI_20240418185805_0190_Z',
                'DJI_20240418185807_0191_Z',
                'DJI_20240418185809_0192_Z',
                'DJI_20240418185811_0193_Z',
                'DJI_20240418185813_0194_Z',
                'DJI_20240418185815_0195_Z']


visibility_dict = {n: get_visible_vertices(n, verbose=False)[0] for n in images_list}
visibility_list = list(visibility_dict.values())

vertex_count = np.zeros(len(pier.vertices), dtype=int)
for vis in visibility_list:
    vertex_count[vis] += 1

In [7]:
import plotly.express as px
def visualize_results_colored(mesh, visible_count, camera_position,):
   
    if len(camera_position.shape)>1:
        camera_position = camera_position.squeeze()
    
    fig = px.scatter_3d(
        x=mesh.vertices[:, 0], y=mesh.vertices[:, 1], z=mesh.vertices[:, 2],
        color=visible_count, size=visible_count+1, opacity=1
    )
 
    fig.show()

In [8]:
visualize_results_colored(pier, vertex_count, center)

In [9]:
p = sum(vertex_count>0)/len(vertex_count)*100
print(f'Percentage of visible vertices {p:.2f}%')

Percentage of visible vertices 1.96%


# Use all poses

In [10]:
import os

# file_format = IMG_FORMAT
# folder = IMG_FOLDER_PATH
file_format = '.json'
folder = 'poses'

def get_names_without_extension(folder_path):
    files = os.listdir(folder_path)
    image_names = [os.path.splitext(f)[0] for f in files if f.lower().endswith(file_format.lower())]
    return image_names

image_names = get_names_without_extension(folder)
print(f'{len(image_names)} poses found')

from random import sample
image_names = sample(image_names, 200)

5689 poses found


In [11]:
visibility_dict = {}

for n in image_names:
    try:
        visibility_dict[n] = get_visible_vertices(n, verbose=False)[0]
    except:
        continue

vertex_count = np.zeros(len(pier.vertices), dtype=int)
empty = 0
for name in visibility_dict:
    if len(visibility_dict[name]) > 0:
        vertex_count[visibility_dict[name]] += 1
    else:
        print(f'{name} view is empty')
        empty += 1
print(f"{empty} poses without visible vertices")

DJI_20240321165049_0074_Z view is empty
DJI_20240321165313_0112_Z view is empty
DJI_20240417155152_0168_Z view is empty
DJI_20240417103019_0196_Z view is empty
DJI_20240417162343_0171_Z view is empty
DJI_20240417105354_0116_Z view is empty
DJI_20240321140506_0022_Z view is empty
DJI_20240417105517_0125_Z view is empty
DJI_20240417155556_0202_Z view is empty
DJI_20240321140940_0076_Z view is empty
DJI_20240417105323_0109_Z view is empty
DJI_20240321133958_0108_Z view is empty
DJI_20240417112055_0016_Z view is empty
DJI_20240417100926_0019_Z view is empty
DJI_20240321134355_0140_Z view is empty
DJI_20240417105522_0126_Z view is empty
DJI_20240321134721_0196_Z view is empty
17 poses without visible vertices


In [12]:
visualize_results_colored(pier, vertex_count, center)

In [13]:
p = np.count_nonzero(vertex_count)/len(vertex_count)*100
print(f'Percentage of visible vertices {p:.2f}%')

Percentage of visible vertices 17.17%


### Here we can see, that most of the vertices are not visible with out method

In [14]:
def greedy_set_cover(images_dict, all_points):
    # Will work only if all vertices are covered
    uncovered = set(all_points)
    cover = []
    images_set = {k: set(v) for k, v in images_dict.items()}
    while uncovered:
        # Find the image that covers the most uncovered points
        best_image = max(images_set, key=lambda x: len(images_set[x] & uncovered))
        # Add the best image to our cover
        cover.append(best_image)
        uncovered -= images_set[best_image]

    return cover

# will not stop
# minimal_cover = greedy_set_cover(visibility_dict, np.arange(len(pier.vertices)))
# print(f"Minimal set of images: {minimal_cover}")
# print(f"Number of images in the cover: {len(minimal_cover)}")

### Solve coverage of all points is not possible with our data

In [15]:
def greedy_max_coverage(images_dict, all_points, N):
    uncovered = set(all_points)
    cover = []
    covered_points = set()
    images_set = {k: set(v) for k, v in images_dict.items()}

    for _ in range(N):
        if not uncovered:
            break
        # Find the image that covers the most uncovered points
        best_image = max(images_set, key=lambda x: len(images_set[x] & uncovered))
        cover.append(best_image)
        newly_covered = images_set[best_image] & uncovered
        covered_points.update(newly_covered)
        uncovered -= newly_covered

    coverage_percentage = (len(covered_points) / len(all_points)) * 100

    return cover, covered_points, coverage_percentage

In [16]:
N = 20

best_cover, covered_points, coverage_percentage = greedy_max_coverage(visibility_dict, np.arange(len(pier.vertices)), N)

print(f"Best set of {N} images: {best_cover}")
print(f"Covered points: {covered_points}")
print(f"Coverage percentage: {coverage_percentage:.2f}%")

Best set of 20 images: ['DJI_20240322095325_0425_W', 'DJI_20240322095744_0471_Z', 'DJI_20240322100011_0524_Z', 'DJI_20240322095141_0378_W', 'DJI_20240419124019_0074_Z', 'DJI_20240418190128_0259_Z', 'DJI_20240322095712_0455_W', 'DJI_20240419123919_0049_Z', 'DJI_20240322094042_0196_Z', 'DJI_20240419145242_0076_Z', 'DJI_20240419122150_0138_Z', 'DJI_20240419145430_0117_Z', 'DJI_20240419122433_0198_Z', 'DJI_20240418185749_0182_Z', 'DJI_20240322094731_0283_W', 'DJI_20240322095335_0430_Z', 'DJI_20240322120027_0373_W', 'DJI_20240418190402_0312_Z', 'DJI_20240322093955_0173_W', 'DJI_20240419123233_0381_Z']
Covered points: {12289, 8196, 4100, 6, 12295, 4104, 11, 8204, 12301, 4110, 4114, 12307, 20, 8210, 22, 12312, 25, 4121, 12315, 28, 8221, 8217, 4126, 34, 4132, 4137, 41, 8233, 47, 8240, 4143, 52, 8245, 4150, 8247, 57, 8253, 8260, 71, 4168, 8264, 77, 8270, 4175, 82, 4180, 86, 8281, 4186, 93, 8286, 99, 8294, 4199, 107, 8302, 4208, 112, 8308, 118, 4214, 120, 8314, 4223, 8320, 127, 130, 4229, 8326, 

coverage of 15.58% instead of 17.03% with 10x less images 

### A working solution for incomplete mesh coverage